# Exercice 3: asymetric cryptography algorithm

This session is based on the book: **Cracking Codes with Python - An Introduction to Building and Breaking Ciphers**

by Al Sweigart
January 2018, 416 pp.
ISBN-13:9781593278229

All codes (BSD Licensed) are available at https://nostarch.com/crackingcodes

In [ ]:
import random, sys, os, math

# Part 1: Prime numbers and modular inverse

## 1.1. Computation of Greatest Common Divisor of a and b using Euclid's Algorithm

In [ ]:
def gcd(a, b):
    # Return the Greatest Common Divisor of a and b using Euclid's Algorithm
    while a != 0:
        a, b = b % a, a
    return b

Check the Euclid's Algorithm and try different values

In [ ]:
#This is an example
gcd(150,3)
gcd(15,10)

5

## 1.2. Computation of the modular inverse of a % m

In [ ]:
def findModInverse(a, m):
    # Return the modular inverse of a % m, which is
    # the number x such that a*x % m = 1

    if gcd(a, m) != 1:
        return None # No mod inverse exists if a & m aren't relatively prime.

    # Calculate using the Extended Euclidean Algorithm:
    u1, u2, u3 = 1, 0, a
    v1, v2, v3 = 0, 1, m
    while v3 != 0:
        q = u3 // v3 # Note that // is the integer division operator
        v1, v2, v3, u1, u2, u3 = (u1 - q * v1), (u2 - q * v2), (u3 - q * v3), v1, v2, v3
    return u1 % m

Try different values and verify the result

In [ ]:
#This is an example
findModInverse(4, 7)

2

## 1.3. Algorithms to verify if a number is a prime number

In [ ]:
import math
def isPrimeTrialDiv(num):
    # Returns True if num is a prime number, otherwise False.

    # Uses the trial division algorithm for testing primality.

    # All numbers less than 2 are not prime:
    if num < 2:
        return False

    # See if num is divisible by any number up to the square root of num:
    for i in range(2, int(math.sqrt(num)) + 1):
        if num % i == 0:
            return False
    return True

Check the algorithm and try different values

In [ ]:
#This is an example
isPrimeTrialDiv(17)

True

In [ ]:
def primeSieve(sieveSize):
    # Returns a list of prime numbers calculated using
    # the Sieve of Eratosthenes algorithm.

    sieve = [True] * sieveSize
    sieve[0] = False # Zero and one are not prime numbers.
    sieve[1] = False

    # Create the sieve:
    for i in range(2, int(math.sqrt(sieveSize)) + 1):
        pointer = i * 2
        while pointer < sieveSize:
            sieve[pointer] = False
            pointer += i

    # Compile the list of primes:
    primes = []
    for i in range(sieveSize):
        if sieve[i] == True:
            primes.append(i)

    return primes

Try different values

In [ ]:
#This is an example
n = primeSieve(200)
print(n)


[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199]


## 1.4. Rabin-Miller Algorithm

In [ ]:
import random
def rabinMiller(num):
    # Returns True if num is a prime number.
    if num % 2 == 0 or num < 2:
        return False # Rabin-Miller doesn't work on even integers.
    if num == 3:
        return True
    s = num - 1
    t = 0
    while s % 2 == 0:
        # Keep halving s until it is odd (and use t
        # to count how many times we halve s):
        s = s // 2
        t += 1
    for trials in range(5): # Try to falsify num's primality 5 times.
        a = random.randrange(2, num - 1)
        v = pow(a, s, num)
        if v != 1: # (This test does not apply if v is 1.)
            i = 0
            while v != (num - 1):
                if i == t - 1:
                    return False
                else:
                    i = i + 1
                    v = (v ** 2) % num
    return True

### Question 1: How works the algorithm? Explain the code.

in the first part of the algorithm, we find u, r variables for the theorem by halving the number until we find an odd one. The number of halvings is the power of 2.
In the second part that is repeated multiples times to increase confidence, we try to find an a that satisfies the theorem by randomly picking a value and iteratively checking if the hyphotesis hold.

We fist calculate a^r mod n and check if the result is one. If that's the case the test automatically fails (the number is not a composite).
Otherwise, we try to find an i that invalidates the second part of the theorem by iterating over the values until t-1 (if we get until the end of the j definition it means that the number is definitely composite) or until the remainder satisfies the conditionm in which case it could be prime.

In [ ]:
#This is an example
rabinMiller(37)

True

## 1.5. Optimized prime number verifier algorithm

In [ ]:
# Most of the time we can quickly determine if num is not prime
# by dividing by the first few dozen prime numbers. This is quicker
# than rabinMiller(), but does not detect all composites.
LOW_PRIMES = primeSieve(100)
print(LOW_PRIMES)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]


In [ ]:
def isPrime(num):
    # Return True if num is a prime number. This function does a quicker
    # prime number check before calling rabinMiller().
    if (num < 2):
        return False # 0, 1, and negative numbers are not prime.
    # See if any of the low prime numbers can divide num:
    for prime in LOW_PRIMES:
        if (num == prime):
            return True
    for prime in LOW_PRIMES:
        if (num % prime == 0):
            return False
    # If all else fails, call rabinMiller() to determine if num is a prime:
    return rabinMiller(num)

Check the algorithm and try different values

In [ ]:
# This is an example
isPrime(870780)

False

## 1.6. Large prime number generation

In [ ]:
def generateLargePrime(keysize=1024):
    # Return a random prime number that is keysize bits in size.
    i=0
    while True:
        num = random.randrange(2**(keysize-1), 2**(keysize))
        #print('random number', i, hex(num))
        i = i + 1
        if isPrime(num):
            return num

### Question 2: Explain the algorithm.

You pick a number, and then we check if it is a prime number or not. If the number is prime, we return the prime number. Otherwise, we pcik another number.

In [ ]:
key = generateLargePrime(2048)
print(hex(key))

0xa0b3065a4a50ac972343c8cb2fcf868b6abfd0e3a7a94fe0d2c9779598a85722469e11829f1a4c33992a2fe9518f94c46b29222f0e7d543bc5a438fb6670658159af97c787f62195b3569272487a538a0298238ee861469aaee0cc6736ff0e5faf39b8cebea5a4af31886009623a0eb4a779a5669dd970d17e4ce15fe05e7d5923a0857cd317503ad4d439e956434b86ca8e22b996751451dc75b3ca58af56bcc9e032494f7c0075752c4a6855391c60c85f64e65e2d1e1b22d61e12313ad57cbca3a8557a4b465f6a3e03f81da6c9fff95a69b94728867f7032360e4d35aa2bff90a4a2656a738f978e1d5b175d13a473ae3e954626a2442eea3cd85dc91fe5


# Part 2: RSA Algorithm

## 2.1. RSA public key and private key generation

In [ ]:
def generateKey(keySize):
    # Creates a public/private keys keySize bits in size.
    p = 0
    q = 0
    # Step 1: Create two prime numbers, p and q. Calculate n = p * q.
    print('Generating p & q primes...')
    while p == q:
        p = generateLargePrime(keySize)
        q = generateLargePrime(keySize)
    n = p * q
    phi = (p - 1) * (q - 1)
    # Step 2: Create a number e that is relatively prime to (p-1)*(q-1):
    print('Generating e that is relatively prime to (p-1)*(q-1)...')
    e = 2
    while gcd(e, phi) != 1:
        # Keep trying random numbers for e until one is valid:
        e = random.randrange(2 ** (keySize - 1), 2 ** (keySize))

    # Step 3: Calculate d, the mod inverse of e:
    print('Calculating d that is mod inverse of e...')
    d = findModInverse(e, phi)

    publicKey = (n, e)
    privateKey = (n, d)

    return (publicKey, privateKey)

### Question 3: Complete the generateKey function and explain your solution.

In [ ]:
# This is an example
publicKey, privateKey = generateKey(128)
print('Public key:', publicKey)
print('Private key:', privateKey)

Generating p & q primes...
Generating e that is relatively prime to (p-1)*(q-1)...
Calculating d that is mod inverse of e...
Public key: (63653797369212215818720574312125881740903419046890298031690795312405481479437, 300960354809865605716501929166901007619)
Private key: (63653797369212215818720574312125881740903419046890298031690795312405481479437, 61985217294012857569975265915196909516318592037574338776693907260740648942247)


## 2.2. Storing public and private keys into files

In [ ]:
def makeKeyFiles(name, keySize):
    # Creates two files 'x_pubkey.txt' and 'x_privkey.txt' (where x
    # is the value in name) with the n,e and d,e integers written in
    # them, delimited by a comma.

    # Our safety check will prevent us from overwriting our old key files:
    if os.path.exists('%s_pubkey.txt' % (name)) or os.path.exists('%s_privkey.txt' % (name)):
        sys.exit('WARNING: The file %s_pubkey.txt or %s_privkey.txt already exists! Use a different name or delete these files and re-run this program.' % (name, name))

    publicKey, privateKey = generateKey(keySize)

    print()
    print('The public key is a %s and a %s digit number.' % (len(str(publicKey[0])), len(str(publicKey[1]))))
    print('Writing public key to file %s_pubkey.txt...' % (name))
    fo = open('%s_pubkey.txt' % (name), 'w')
    fo.write('%s,%s,%s' % (keySize, publicKey[0], publicKey[1]))
    fo.close()

    print()
    print('The private key is a %s and a %s digit number.' % (len(str(publicKey[0])), len(str(publicKey[1]))))
    print('Writing private key to file %s_privkey.txt...' % (name))
    fo = open('%s_privkey.txt' % (name), 'w')
    fo.write('%s,%s,%s' % (keySize, privateKey[0], privateKey[1]))
    fo.close()

Here we created two files for storing the private key and public key. We have the keysize, private key and public key.

### Question 4: Explain the makeKeyFiles function

In [ ]:
# Create a public/private keypair with 1024 bit keys:
print('Making key files...')
makeKeyFiles('testt_key', 1024)
print('Key files made.')


Making key files...
Generating p & q primes...
Generating e that is relatively prime to (p-1)*(q-1)...
Calculating d that is mod inverse of e...

The public key is a 617 and a 309 digit number.
Writing public key to file testt_key_pubkey.txt...

The private key is a 617 and a 309 digit number.
Writing private key to file testt_key_privkey.txt...
Key files made.


## 2.3. RSA encryption

In [ ]:
# IMPORTANT: The block size MUST be less than or equal to the key size!
# (Note: The block size is in bytes, the key size is in bits. There
# are 8 bits in 1 byte.)

DEFAULT_BLOCK_SIZE = 128 # 128 bytes
BYTE_SIZE = 256 # One byte has 256 different values.

def getBlocksFromText(message, blockSize=DEFAULT_BLOCK_SIZE):
    # Converts a string message to a list of block integers. Each integer
    # represents 128 (or whatever blockSize is set to) string characters.

    messageBytes = message.encode('ascii') # convert the string to bytes

    blockInts = []
    for blockStart in range(0, len(messageBytes), blockSize):
        # Calculate the block integer for this block of text
        blockInt = 0
        for i in range(blockStart, min(blockStart + blockSize, len(messageBytes))):
            blockInt += messageBytes[i] * (BYTE_SIZE ** (i % blockSize))
        blockInts.append(blockInt)
    return blockInts


def encryptMessage(message, key, blockSize=DEFAULT_BLOCK_SIZE):
    # Converts the message string into a list of block integers, and then
    # encrypts each block integer. Pass the PUBLIC key to encrypt.
    encryptedBlocks = []
    n, e = key

    for block in getBlocksFromText(message, blockSize):
        # ciphertext = plaintext ^ e mod n

        # complete the code to perform the modular exponentiation (encryption)
        # ...
        ciphertext = pow(block, e, n)
        encryptedBlocks.append(ciphertext)

    return encryptedBlocks


def readKeyFile(keyFilename):
    # Given the filename of a file that contains a public or private key,
    # return the key as a (n,e) or (n,d) tuple value.
    fo = open(keyFilename)
    content = fo.read()
    fo.close()
    keySize, n, EorD = content.split(',')
    return (int(keySize), int(n), int(EorD))


def encryptAndWriteToFile(messageFilename, keyFilename, message, blockSize=DEFAULT_BLOCK_SIZE):
    # Using a key from a key file, encrypt the message and save it to a
    # file. Returns the encrypted message string.
    keySize, n, e = readKeyFile(keyFilename)

    # Check that key size is greater than block size.
    if keySize < blockSize * 8: # * 8 to convert bytes to bits
        sys.exit('ERROR: Block size is %s bits and key size is %s bits. The RSA cipher requires the block size to be equal to or greater than the key size. Either decrease the block size or use different keys.' % (blockSize * 8, keySize))

    # Encrypt the message
    encryptedBlocks = encryptMessage(message, (n, e), blockSize)

    # Convert the large int values to one string value.
    for i in range(len(encryptedBlocks)):
        encryptedBlocks[i] = str(encryptedBlocks[i])
    encryptedContent = ','.join(encryptedBlocks)

    # Write out the encrypted string to the output file.
    encryptedContent = '%s_%s_%s' % (len(message), blockSize, encryptedContent)
    fo = open(messageFilename, 'w')
    fo.write(encryptedContent)
    fo.close()
    # Also return the encrypted string.
    return encryptedContent

### Question 5: Complete the code and explain the encryption process.

In [ ]:
# Runs a test that encrypts a message to a file or decrypts a message
# from a file.
filename = 'encrypted_file.txt' # the file to write to/read from

message = '''"Journalists belong in the gutter because that is where the ruling classes throw their guilty secrets." -Gerald Priestland "The Founding Fathers gave the free press the protection it must have to bare the secrets of government and inform the people." -Hugo Black'''
pubKeyFilename = 'testt_key_pubkey.txt'
print('Encrypting and writing to %s...' % (filename))
encryptedText = encryptAndWriteToFile(filename, pubKeyFilename, message)

print('Encrypted text:')
print(encryptedText)

Encrypting and writing to encrypted_file.txt...
Encrypted text:
262_128_2928253126428522715982838705821322546623010892756310452820821685488590818826487149955793029318373473345096195757117610739596969071342383562473520862785263440964628034362522576341809225828310372056191244803567029668659851485434419587613908532349027035760087430013153156739511006147183887646298067380508799929568699867206939836574977273708507383273044702146561562308058262872850686464869262841221213886848291306469297233543198278325412341784255110373634972835188563203507629965016866940049285697161779834423982839905657499186670308555770228495619332151377388339755657798375984323219444752966663080081884016189902850105,83310360186735818144117942446141208741645372877824129996326079280765475200798909801930640426228999401570098012360050591890725530623237174096399527719744102190742634354870569946524715606488372394525888796100672679110043947102659395271232046192488285666296987238797876039866686710803055662225399739313965231595599

### Question 6: Instead of writing the message, propose a new function that read a file containing the plaintext (message) and save it in the message variable.

In [ ]:
import unicodedata, string, re, random, math
def convert(s):
    "Convert the string 's', supposed to be encoded in UTF-8, into a very simple one."
    tmp1 = unicodedata.normalize('NFKD', s) # decode the input string
    tmp2 = tmp1.encode('ASCII', 'ignore').decode('utf8') # convert it to ASCII (remove accents) and to UTF8
    tmp3 = re.sub('[\W\d_]+', ' ', tmp2) # convert non-[a-zA-Z] chars into 1 space
    tmp4 = re.sub('^\s*|\s*$', '', tmp3) # remove leading/trailing spaces
    tmp5 = tmp4.upper() # convert to upcase
    return tmp5
def read_file(filename):
    "Read the text in 'filename' and convert it into a very simple, but possibly large, string."
    res = []
    with open(filename, 'r', encoding='utf8') as f:
        for l in f:
            line = convert(l)
            if line: # do not append empty lines
                res.append(line)
    return ' '.join(res)
def read_file_encrypt(source_file, dest_file, pub_key_path):
  filename = dest_file # the file to write to/read from
  message = read_file(source_file)
  pubKeyFilename = pub_key_path
  encryptedText = encryptAndWriteToFile(filename, pubKeyFilename, message)
filename = "test_encrypt.txt"
#read_file_encrypt("testt_file.txt", filename, "test_key_pubkey.txt")

## 2.4. RSA decryption

In [ ]:
# IMPORTANT: The block size MUST be less than or equal to the key size!
# (Note: The block size is in bytes, the key size is in bits. There
# are 8 bits in 1 byte.)

DEFAULT_BLOCK_SIZE = 128 # 128 bytes
BYTE_SIZE = 256 # One byte has 256 different values.

def getTextFromBlocks(blockInts, messageLength, blockSize=DEFAULT_BLOCK_SIZE):
    # Converts a list of block integers to the original message string.
    # The original message length is needed to properly convert the last
    # block integer.
    message = []
    for blockInt in blockInts:
        blockMessage = []
        for i in range(blockSize - 1, -1, -1):
            if len(message) + i < messageLength:
                # Decode the message string for the 128 (or whatever
                # blockSize is set to) characters from this block integer.
                asciiNumber = blockInt // (BYTE_SIZE ** i)
                #print(chr(asciiNumber))
                #blockInt = blockInt % (BYTE_SIZE ** i)
                blockInt = blockInt % (pow(BYTE_SIZE, i))
                blockMessage.insert(0, chr(asciiNumber))
        message.extend(blockMessage)
    return ''.join(message)

def decryptMessage(encryptedBlocks, messageLength, key, blockSize=DEFAULT_BLOCK_SIZE):
    # Decrypts a list of encrypted block ints into the original message
    # string. The original message length is required to properly decrypt
    # the last block. Be sure to pass the PRIVATE key to decrypt.
    decryptedBlocks = []
    n, d = key
    for block in encryptedBlocks:
        # plaintext = ciphertext ^ d mod n

        # complete the code to perform the modular exponentiation (decryption)
        # ...
        plaintext = pow(block, d, n)
        decryptedBlocks.append(plaintext)
    return getTextFromBlocks(decryptedBlocks, messageLength, blockSize)

def readFromFileAndDecrypt(messageFilename, keyFilename):
    # Using a key from a key file, read an encrypted message from a file
    # and then decrypt it. Returns the decrypted message string.
    keySize, n, d = readKeyFile(keyFilename)


    # Read in the message length and the encrypted message from the file.
    fo = open(messageFilename)
    content = fo.read()
    messageLength, blockSize, encryptedMessage = content.split('_')
    messageLength = int(messageLength)
    blockSize = int(blockSize)

    # Check that key size is greater than block size.
    if keySize < blockSize * 8: # * 8 to convert bytes to bits
        sys.exit('ERROR: Block size is %s bits and key size is %s bits. The RSA cipher requires the block size to be equal to or greater than the key size. Did you specify the correct key file and encrypted file?' % (blockSize * 8, keySize))

    # Convert the encrypted message into large int values.
    encryptedBlocks = []
    for block in encryptedMessage.split(','):
        encryptedBlocks.append(int(block))

    # Decrypt the large int values.
    return decryptMessage(encryptedBlocks, messageLength, (n, d), blockSize)


### Question 7: Complete the code and explain the decryption process.

In [ ]:
filename = 'encrypted_file.txt'
privKeyFilename = 'test_key_privkey.txt'
print('Reading from %s and decrypting...' % (filename))

decryptedText = readFromFileAndDecrypt(filename, privKeyFilename)

print('Decrypted text:')
print(decryptedText)

Reading from encrypted_file.txt and decrypting...
Decrypted text:


NameError: name 'decryptedText' is not defined

In [ ]:
keyFilename='testt_key'
pubKeyFilename = keyFilename+'_pubkey.txt'
makeKeyFiles(keyFilename, 1024)

SystemExit: WARNING: The file testt_key_pubkey.txt or testt_key_privkey.txt already exists! Use a different name or delete these files and re-run this program.

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### Question 8: Instead of printing the message, propose a new function that writes a file containing the plaintext.

# Part 3: Use of pycryptodome librairy

## 3.1. RSA

Explore the pycryptodome librairy https://pycryptodome.readthedocs.io/en/latest/index.html. Test possible functions to generate keys and to use RSA.

In [ ]:
%pip install pycryptodome
from Crypto.Cipher import PKCS1_OAEP
from Crypto.PublicKey import RSA
keypair = RSA.generate(2048)
public_key = keypair.publickey()

message = b'You can attack now!'

cipher = PKCS1_OAEP.new(public_key)

ciphertext = cipher.encrypt(message)
print(ciphertext)
cipher = PKCS1_OAEP.new(keypair)

message = cipher.decrypt(ciphertext)
print(message)

b"\x15zS\xfeP\x8b\n\xa7\x10??\xcf\x8f\x9a\x0c[\xc3&\xec\xd7\xb2\xeb\x8e\xa5\x81\xceT\x97q\x91\x18?:a\xa3\xf15]\x0e\xc9h'\x9d\xc0\xd2\xd0\x89\xba\xb6\x11\x12\x02\xff\xd6_\xbf.3\r\xa3\xb9R'\xe217~7\xf6\xc4\xc7\x19l\xe8\x83E\x80\x85D7\xd7%\x10\xc4\xaf\xafs\xcc@\xc4l\xe9\xbb@\xa3\xd0=\xa3\x9d\x9d'\xc2\x9a^\xff\\M__:\x05\xa4\x1bTWE\x86T\xda\x0e\x9e]\x0bh\xfdZ\x1e\x03R\xd9\xcb\xf0\x84R\xa9\xebM\xad\xe0\xb0\xcf]B\x8f\x87x\x1f\xab1\xeaA\x94\x95\xe7\xa1\x8ao\x99\x1ds\xd4\xc4\xcd\x1d\x87\x814\xa8]\xbceX\x8f\xe7a\xd04\xd9\xd8\xb6\x13v\xc0\nE\x9dh\xa9\x95\x8d\xe4B\x0e\xd7\xb0'IuX\xd3i\x98\xa7\xec\xe9\xc4\x99\xddL\xc47Wj\xdeod\x10\xe8V\xcc\x0e0\x98 ]\xb3\xea\x90\xb4\xee\x90z\xcf\xd69\xb5e\xd0\x12*\xa1_\xa3\xe2!\x90\x8ftL6m\x02\x94Y\x88\xbb"
b'You can attack now!'


## 3.2. ECC

### Question 9: Run the cells below and explain how the Digital Signature Algorithm works. Modify the message or the signature and check if the algorithm detects it.

In [ ]:
from Crypto.Hash import SHA256
from Crypto.PublicKey import ECC
from Crypto.Signature import DSS

key = ECC.generate(curve='P-256')

f = open('key.pem','wt')
f.write(key.export_key(format='PEM'))
f.close()

f = open('key.pem','rt')
pubkey = ECC.import_key(f.read())

In [ ]:
print(key)

EccKey(curve='NIST P-256', point_x=86745050085852438109123933877965600484811125816749613374989492034166390628790, point_y=98557479687999984060834398560939965798377971509922269117559478357419854100460, d=24001350477956028381032824165271577897559602933468267857897472838828509307449)


In [ ]:
my_message = b'I give my permission to order #4355'
privkey = ECC.import_key(open('key.pem').read())
h = SHA256.new(my_message)
print(h.hexdigest())
signer = DSS.new(privkey, 'fips-186-3')
sender_signature = signer.sign(h)

4076b23570ab85c5772b2070b842c4137a815920b8505724efde7ae94c278d13


In [ ]:
#print(str(sender_signature))
#print(type(sender_signature))
#for i in range (0,len(sender_signature)):
#    print(hex(sender_signature[i]))
my_signature = 0
i = 64
for byte in sender_signature:
    Number = byte * pow(2,i*8)
    my_signature = my_signature + Number
    i = i - 1
print(hex(my_signature))

0x26fe74b01df983eac7f1dac2bce761a124df00b00c207a6c1bedcca8347d84cf6ba189f6c92d04ecbf734c99af5225e06b7cd272965d942f0b5a162e079ef68200


In [ ]:
received_signature = sender_signature
received_message = b'I give my permission to order #4355'

pubkey = ECC.import_key(open('key.pem').read())
h_received = SHA256.new(received_message)

verifier = DSS.new(pubkey, 'fips-186-3')
try:
    verifier.verify(h_received, received_signature)
    print("The message is authentic.")
except ValueError:
    print("The message is NOT authentic.")

The message is authentic.
